In [1]:
from pymongo import MongoClient
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from functools import reduce
import operator
import json
import re
from geopy.geocoders import Nominatim

In [2]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd

In [5]:
import ast

In [3]:
df = pd.read_csv('../data/data_tripadvisor.csv', index_col=0)
df.dropna(how = "all", axis = 0, inplace = True)

In [4]:
df.head()

,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3
indice,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,#549 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España"
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,#34 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España"
95.0,Casa Labra,"Calle Tetuan 12, 28013 Madrid España",Bar,#650 de 10.414 restaurantes en Madrid,€,Calle Tetuan 12,Madrid España,Calle Tetuan 12,"Calle Tetuan 12, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 12, Madrid España"
134.0,Parrilla El Gaucho,"Calle Tetuan 34, 28013 Madrid España",Restaurante de carne,#311 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Tetuan 34,Madrid España,Calle Tetuan 34,"Calle Tetuan 34, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 34, Madrid España"
279.0,Santa Eulalia Boulangerie,"Calle Espejo 12, 28013 Madrid España",Confiterías/panaderías,N.º 23 de 477 cafeterías y teterías en Madrid,€€ - €€€,Calle Espejo 12,Madrid España,Calle Espejo 12,"Calle Espejo 12, Madrid España","(40.1384846, -3.4252641)","Calle Espejo 12, Madrid España"


In [10]:
df.dtypes

name           object
address        object
type           object
ranking        object
price          object
direccion      object
comunidad      object
street         object
street_2       object
coordinates    object
street_3       object
dtype: object

In [8]:
def ast_literal (x):
    try:
        return ast.literal_eval(x)
    except:
        return x

In [9]:
df.coordinates = df.coordinates.apply(ast_literal)

In [12]:
def turn_around(x):
    b = (x[1], x[0])
    return list(b)

In [13]:
df["coordinates_v2"] = df.coordinates.apply(turn_around)

In [14]:
df.head()

,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,coordinates_v2
indice,,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,#549 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España","[-3.606505, 40.028851]"
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,#34 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España","[-3.5908706, 40.0316837]"
95.0,Casa Labra,"Calle Tetuan 12, 28013 Madrid España",Bar,#650 de 10.414 restaurantes en Madrid,€,Calle Tetuan 12,Madrid España,Calle Tetuan 12,"Calle Tetuan 12, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 12, Madrid España","[-3.6142713, 40.0395885]"
134.0,Parrilla El Gaucho,"Calle Tetuan 34, 28013 Madrid España",Restaurante de carne,#311 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Tetuan 34,Madrid España,Calle Tetuan 34,"Calle Tetuan 34, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 34, Madrid España","[-3.6142713, 40.0395885]"
279.0,Santa Eulalia Boulangerie,"Calle Espejo 12, 28013 Madrid España",Confiterías/panaderías,N.º 23 de 477 cafeterías y teterías en Madrid,€€ - €€€,Calle Espejo 12,Madrid España,Calle Espejo 12,"Calle Espejo 12, Madrid España","(40.1384846, -3.4252641)","Calle Espejo 12, Madrid España","[-3.4252641, 40.1384846]"


In [16]:
type(df.coordinates_v2[0])

list

In [18]:
def type_point(x):
    return {"type":"Point", "coordinates": [x[1], x[0]]}

In [19]:
df["type_point"] = df["coordinates"].apply(type_point)

In [21]:
df.head(2)

,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,coordinates_v2,type_point
indice,,,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,#549 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España","[-3.606505, 40.028851]","{'type': 'Point', 'coordinates': [-3.606505, 4..."
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,#34 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España","[-3.5908706, 40.0316837]","{'type': 'Point', 'coordinates': [-3.5908706, ..."


In [23]:
type(df.type_point[0])

dict

In [24]:
def get_latitude (x):
    return x[0]

In [25]:
def get_longitude (x):
    return x[1]

In [26]:
df["longitude"] = df["coordinates"].apply(get_longitude)

In [27]:
df["latitude"] = df["coordinates"].apply(get_latitude)

In [30]:
df.head(2)

,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,coordinates_v2,type_point,longitude,latitude
indice,,,,,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,#549 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España","[-3.606505, 40.028851]","{'type': 'Point', 'coordinates': [-3.606505, 4...",-3.606505,40.028851
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,#34 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España","[-3.5908706, 40.0316837]","{'type': 'Point', 'coordinates': [-3.5908706, ...",-3.590871,40.031684


In [31]:
def delete_hashtag(x):
    x = str(x).replace('#', "")
    return x

In [32]:
df["ranking"] = df.ranking.apply(delete_hashtag)

In [33]:
def delete_string(x):
    x = str(x).replace('restaurantes en Madrid', "")
    return x

In [34]:
df["ranking"] = df.ranking.apply(delete_string)

In [36]:
df.head(2)

,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,coordinates_v2,type_point,longitude,latitude
indice,,,,,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,549 de 10.409,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España","[-3.606505, 40.028851]","{'type': 'Point', 'coordinates': [-3.606505, 4...",-3.606505,40.028851
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,34 de 10.414,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España","[-3.5908706, 40.0316837]","{'type': 'Point', 'coordinates': [-3.5908706, ...",-3.590871,40.031684


In [37]:
df.to_csv('tripadvisor_latlon.csv')

## Exportar a Mongo

In [38]:
client = MongoClient("localhost:27017")

In [39]:
db = client.get_database("PROJECT")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PROJECT')

In [40]:
collection = db["Restaurants"]

In [ ]:
collection.create_index([("type_point_", "2dsphere")])

In [41]:
data = df.to_dict(orient='records')

In [44]:
df.type_point[0]

{'type': 'Point', 'coordinates': [-3.679035, 40.4243394]}

In [45]:
collection.insert_many(data)

In [ ]:
ubicacion = input("¿Dónde estás?: ")
print(ubicacion)

In [ ]:
tipo = input("¿Qué tipo de comida te apetece?: ")
print(tipo)

In [ ]:
distance = int(input("¿Cuántos metros quieres desplazarte?: "))
print(distance)

In [ ]:
def get_coordenadas(address):
    """
    Esta función saca las coordenadas de la ciudad que le pases.
    Args: una ciudad (string).
    Return: Las coordeandas de la ciudad que le paso como argumento (latitud y longitud).
    """
    geolocator = Nominatim(user_agent="Ana")
    location = geolocator.geocode(query=address, exactly_one=True)
    localizacion = location[1]
    return [localizacion[1], localizacion[0]]

In [ ]:
coordinate = get_coordenadas(ubicacion)

In [ ]:
coordinate

In [ ]:
inicial_lat = coordinate[1]
print(inicial_lat)
inicial_long =  coordinate[0]

In [ ]:
def type_point(lista):
    return {"type":"Point", "coordinates": lista}

In [ ]:
db

## Restaurantes a 500 m

In [ ]:
restaurants = db.get_collection("Restaurants")

In [ ]:
all_restaurants = {"coordinates_v2": {"$near": {"$geometry":{'type': 'Point', 'coordinates':coordinate},
                                                  "$maxDistance": distance}}}

In [ ]:
(list(restaurants.find(all_restaurants)))

In [ ]:
df_restaurantes= pd.DataFrame(list(restaurants.find(all_restaurants)))

In [ ]:
df_restaurantes

In [ ]:
df_restaurantes.type.unique()

## Restaurantes Mediterraneos

In [ ]:
rest_mediterraneos = {"coordinates_v2": {"$near": {"$geometry":{'type': 'Point', 'coordinates':coordinate},
                                                  "$maxDistance": distance}},'type': "Mediterránea"}

In [ ]:
rest_mediterraneos

In [ ]:
len(list(restaurants.find(query_restaurants)))

In [ ]:
df_medit = pd.DataFrame(list(restaurants.find(query_restaurants)))

In [ ]:
df_medit

## Restaurantes Española

In [ ]:
rest_internacional = {"coordinates_v2": {"$near": {"$geometry":{'type': 'Point', 'coordinates':coordinate},
                                                  "$maxDistance": distance}},'type': "Internacional"}

In [ ]:
rest_internacional

In [ ]:
len(list(restaurants.find(rest_internacional)))

In [ ]:
df_intern = pd.DataFrame(list(restaurants.find(rest_internacional)))

In [ ]:
df_intern

## Maps

In [ ]:
df_restaurantes.head()

In [ ]:
def get_latitude (x):
    return x[0]

In [ ]:
def get_longitude (x):
    return x[1]

In [ ]:
df_restaurantes["latitud"] = df_restaurantes["coordinates"].apply(get_latitude)

In [ ]:
df_restaurantes["longitude"] = df_restaurantes["coordinates"].apply(get_longitude)

In [ ]:
df_restaurantes

In [ ]:
map_1 = folium.Map(location=[inicial_lat,inicial_long], zoom_start=15)

dic = {"location": [inicial_lat, inicial_long], "tooltip": "ubicacion"}

icono = Icon(color = "red",
                     prefix="fa",
                     icon="map-marker",
                     icon_color="black")

mark = Marker(**dic, icon=icono)
mark.add_to(map_1)

In [ ]:
map_1

In [ ]:
df_restaurantes.type.unique()

In [ ]:
for i,row in df_restaurantes.iterrows():

    #dic = {"location": [row["latitud"], row["longitud"]], "tooltip": row["nombre"]}
    dic = {"location": [row["latitud"], row["longitude"]], "tooltip": row["name"]}

    if row["type"] == "Mediterránea":
        icono = Icon(color = "green",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")

    elif row["type"] == 'Italiana':
        icono = Icon(color = "lightblue",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")

        
    elif row["type"] == "Internacional":        
        icono = Icon(color = "orange",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
    elif row["type"] == "Confiterías/panaderías":        
        icono = Icon(color = "gray",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
    elif row["type"] == "Japonesa":        
        icono = Icon(color = "cadetblue",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
           
    elif row["type"] == "Bar":        
        icono = Icon(color = "lightgreen",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
           
    elif row["type"] == "India":        
        icono = Icon(color = "lightblue",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
                 
           
    elif row["type"] == "Mexicana":        
        icono = Icon(color = "pink",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
           
    elif row["type"] == "Pub con cerveza artesanal":        
        icono = Icon(color = "purple",
                     prefix="fa",
                     icon="circle",
                     icon_color="black")
           
    
    mark = Marker(**dic, icon=icono)
    mark.add_to(map_1)   

In [ ]:
map_1

1- Input preguntando donde estas
2- Input preguntando que radio quieren
3- Funcion que pasa de direccion a coordenadas (coordenadas al reves)
4- Funcion que me haga una query near a moongo y me devuelva todos los restaurantes en el radio  




streamlit : parte del csv, estoy en tal sitio y quiero tal tipo de comida en esta distancia.

- tendre una funcion que haga la query y saque un pandas. support
- tendre una función que haga un mapa con un pd. return map. spport


- df = función(input del usuario)
- map (df)




mañana hay que tener la presentación.

1- mapas (aqui va el input del usuario)
para la distancia hago un sidebar. esta en iris_prediction/src/ support.py /

2 - graficos


3- diccionario map sobre el pd.





